In [ ]:
# Adam Patyk
# Clemson University
# MS Thesis: Daily Pattern Classifier
# Summer 2021

# LoadFiles.ipynb
# Purpose: Consolidate daily sample data from text files into NumPy arrays

import os
import tensorflow as tf
import numpy as np
import glob
import multiprocessing as mp
from tqdm import tqdm
from datetime import datetime

### LOAD FILES FROM MULTIPLE DIRECTORIES IN PARALLEL

W = [2, 4, 6, 8, 10]
samples, labels = [], []

# reader function for parallelized text file reading
def reader(window_size):
    data  = []
    for filename in tqdm(glob.glob(os.path.join(data_dir + f'W{window_size}/', '*.txt'))):
        d = np.loadtxt(filename)
        f = np.full((len(d), 1), int(filename[-12:-9]))
        data.append(np.hstack((f, d)))
    return data
    
data_dir = 'samples/'

start_time = datetime.now()

print(f'Loading data in parallel...', flush=True) # buffer flush needed with tqdm
# read text files in parallel
pool = mp.Pool(len(W))
data_ls = pool.map(reader, W)

end_time = datetime.now()
print(f'Duration: {end_time - start_time}', flush=True) # buffer flush needed with tqdm

# arrange data into samples, labels and filenames for saving
all_data = np.hstack(data_ls)
all_samples = [x[:, 2] for x in all_data]
all_labels = [x[:, 1] for x in all_data]
all_filenames = [x[0, 0] for x in all_data]

# save data to numpy arrays
!mkdir -p compressed-samples
np.save('compressed-samples/daily-samples.npy', np.array(all_samples, dtype=object))
np.save('compressed-samples/daily-labels.npy', all_labels)
np.save('compressed-samples/daily-filenames.npy', all_filenames)

print('Done.')

In [ ]:
import os
import numpy as np
import glob
from tqdm import tqdm

### LOAD FILES FROM SINGLE DIRECTORY (SERIAL)

data_dir = 'samples/'

all_data  = []
for filename in tqdm(glob.glob(os.path.join(data_dir, '*.txt'))):
    d = np.loadtxt(filename)
    f = np.full((len(d), 1), int(filename[-12:-9]))
    all_data.append(np.hstack((f, d)))
    
all_samples = [x[:, 2] for x in all_data]
all_labels = [x[:, 1] for x in all_data]
all_filenames = [x[0, 0] for x in all_data]

!mkdir -p compressed-samples
np.save('compressed-samples/daily-samples.npy', np.array(all_samples, dtype=object))
np.save('compressed-samples/daily-labels.npy', all_labels)
np.save('compressed-samples/daily-filenames.npy', all_filenames)